# Join geolocated datasets in LONG format and replace NA's by zero

This is the analysis notebook file for joining and creating long formats of all geolocated datamining output files, based on the EASIN introductions past 2016 up until now. 

### 1. For wikipedia language - based filtering pageviews

In [15]:
#Make sure working directtory is set correctly
import os
print(os.getcwd())

c:\Users\simon\Documents\GitHub\IAScraper\sample\dataprocessing


In [ ]:
#load libraries
import pandas as pd
from datetime import datetime

# Load data

wiki_lang = pd.read_csv("../../species_pageviews_analysis_2016_present.csv")

uni = pd.read_csv("../../UnionList_Species_Traits_85_present.csv")

# Filter introductions from 2016 onwards
intro_year = (
    uni[uni["YEAR"] >= 2016]
    .loc[:, ["SCIENTIFIC_NAME", "COUNTRY", "YEAR", "WIKI NAME", "Group", "Habitat", "EASIN.ID"]]
    .drop_duplicates()
)

# Define language to country mapping
lang2country = {
    "en": ["UK", "IE"], "es": ["ES"], "el": ["EL"], "fr": ["FR", "LU"], "de": ["DE", "AT", "CH"],
    "it": ["IT"], "pt": ["PT"], "nl": ["NL", "BE"], "sv": ["SE"], "da": ["DK"], "fi": ["FI"],
    "cs": ["CZ"], "hr": ["HR"], "hu": ["HU"], "pl": ["PL"], "ro": ["RO"], "sk": ["SK"], "sl": ["SI"],
    "bg": ["BG"], "et": ["EE"], "lv": ["LV"], "lt": ["LT"], "mt": ["MT"],
    "be": ["BE"], "at": ["AT"], "lu": ["LU"], "cy": ["CY"]
}

# Explode language-country mapping into rows
wiki_lang["COUNTRY"] = wiki_lang["Language"].map(lang2country)
wiki_lang = wiki_lang.explode("COUNTRY").dropna(subset=["COUNTRY"])

# Join on WIKI NAME and COUNTRY
wiki_lang["WIKI NAME"] = wiki_lang["Scientific Name"]
wiki_lang2 = wiki_lang.merge(intro_year, how="inner", on=["WIKI NAME", "COUNTRY"])

# Pivot to long format
date_cols = wiki_lang2.columns[wiki_lang2.columns.str.match(r"^\d{8}$")]
wiki_lang_long = (
    wiki_lang2.melt(id_vars=["SCIENTIFIC_NAME", "COUNTRY", "YEAR"], 
             value_vars=date_cols,
             var_name="date", value_name="views")
    .fillna({"views": 0})
)

# Convert date
wiki_lang_long["date"] = pd.to_datetime(wiki_lang_long["date"], format="%Y%m%d", errors="coerce")
wiki_lang_long = wiki_lang_long.dropna(subset=["date"])

# Add platform column
wiki_lang_long["platform"] = "wiki_lang"

# Optional: optimize types
wiki_lang_long["SCIENTIFIC_NAME"] = wiki_lang_long["SCIENTIFIC_NAME"].astype("category")
wiki_lang_long["COUNTRY"] = wiki_lang_long["COUNTRY"].astype("category")
wiki_lang_long["views"] = wiki_lang_long["views"].astype(int)

### 2. For wikipedia geolocated pageviews

In [18]:
import pandas as pd
import pycountry

# Load files
uni = pd.read_csv("../../UnionList_Species_Traits_85_present.csv")
geo_new = pd.read_csv("../../species_pageviews_wiki_geolocated_2023-02-06_now.csv")
geo_old = pd.read_csv("../../species_pageviews_wiki_geolocated_2017-02-09_2023-02-05.csv")

# Filter introductions to YEAR >= 2017
intro_year_geo = (
    uni[uni["YEAR"] >= 2017]
    .loc[:, ["SCIENTIFIC_NAME", "COUNTRY", "YEAR", "Group", "Habitat", "WIKI NAME", "EASIN.ID"]]
    .drop_duplicates()
)

# Combine new + old pageviews
combined_geo = pd.concat([geo_old, geo_new], ignore_index=True)

# Fix EL -> GR and convert ISO2 → full country names
def iso2_to_country(iso2):
    if iso2 == "EL":
        iso2 = "GR"
    try:
        return pycountry.countries.get(alpha_2=iso2).name
    except:
        return None

intro_year_geo["COUNTRY_NAME"] = intro_year_geo["COUNTRY"].map(iso2_to_country)

# Join with combined_geo on Scientific Name + Country
filtered_data = combined_geo.merge(
    intro_year_geo,
    how="inner",
    left_on=["Scientific Name", "Country"],
    right_on=["WIKI NAME", "COUNTRY_NAME"]
)

# Melt to long format using all YYYY-MM-DD columns
date_cols = filtered_data.columns[filtered_data.columns.str.match(r"^\d{4}-\d{2}-\d{2}$")]
long_data_geo = filtered_data.melt(
    id_vars=["SCIENTIFIC_NAME", "Country", "YEAR"],
    value_vars=date_cols,
    var_name="date",
    value_name="views"
).fillna({"views": 0})

# Convert and clean date
long_data_geo["date"] = pd.to_datetime(long_data_geo["date"], errors="coerce")
long_data_geo = long_data_geo.dropna(subset=["date"])
long_data_geo["views"] = long_data_geo["views"].astype(int)

# Add platform label
long_data_geo["platform"] = "wiki_geo"

# Optional: optimize
long_data_geo["SCIENTIFIC_NAME"] = long_data_geo["SCIENTIFIC_NAME"].astype("category")
long_data_geo["Country"] = long_data_geo["Country"].astype("category")
